# 네이버 웹툰

웹툰| https://comic.naver.com/webtoon

- [import](#import)
- [selenium setting](#setting)
- [문제와 해결방법 idea](#문제와-해결방법-idea)
- [연재 웹툰](#연재중인-웹툰)
    * [웹툰 information](#웹툰-information)
    * [에피소드 별 평점](#각-웹툰-에피소드-당-평점)
    * [웹툰 별, 각 에피소드 댓글](#웹툰-별,-각-에피소드-댓글)
        + [중간에 멈추는 경우](#중간에-멈출경우)

# import

In [26]:
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains # 스크롤 다운위해서
from selenium.webdriver.common.keys import Keys
import pyperclip
import time

import warnings
warnings.filterwarnings('ignore')

from selenium.webdriver.support.select import Select

In [27]:
import sys
print(sys.version) # 3.9.12

import selenium
print(selenium.__version__) # 4.2.0

3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
4.2.0


# setting

In [28]:
# 경로 설정
path = 'C:\chromedriver_win32\chromedriver.exe'
url = 'https://comic.naver.com/webtoon'

# 드라이버 설정
def drive_path(x, path):
    global driver # 전역변수 선언하기
    if x == 'chrome':
        driver = webdriver.Chrome(path)
    else:
        print('driver 새로 설정하세요')

# url 설정
def url_move(url):
    if isinstance(url, str):
        driver.get(url)
    else:
        driver.get(str(url))

In [29]:
drive_path('chrome',path)
print('드라이버 설정 완료')
print('-----------------------')

url_move(url)
print('url로 창 열기 완료')
print('-----------------------')
time.sleep(3) # seconds 기다리기

driver.set_window_size(1536, 1024) # 창 사이즈 변경

드라이버 설정 완료
-----------------------
url로 창 열기 완료
-----------------------


---

# 문제와 해결방법 idea

- 기다리면 무료인 작품은 클릭하면 바로 로그인 창이 뜸 → 웹툰에 기다리면 무료 아이콘이 존재할 경우 클릭하지 않고, pass하기 + 성인 제한 걸린 것도 같이 pass 됨 + 신작이라는 아이콘도같이 pass된다는 문제
- 웹툰 전체 에피소드가 무료인 경우도 있지만 일부만 무료이고, 유료인 경우도 있음. 유료인 회차 클릭 시 로그인 창으로 이동 → 유료인 경우, 회차 옆에 아이콘 존재. 있는 경우와 아닌 경우로 나눠서 (우선 다 무료라는 전제로 코드 짠 후, 수정하는 방향으로)
- best 댓글이 없는 경우도 존재
- 전체 댓글 약 200개 수집 시, 댓글이 200개가 안되는 경우 존재 → 전체 댓글 수 추출하여 활용
- 삭제된 댓글 존재

---

# 연재중인 웹툰

요일별로 Top 10 웹툰

In [30]:
global webtoon_info, comment_data, score_data
webtoon_info = pd.DataFrame(columns=['seq','day','title','descript','genre','author','artist','origin','total','star']) # 웹툰 정보
score_data = pd.DataFrame(columns=['title','day','episode','score','date']) # 에피소드 평점
comment_data = pd.DataFrame(columns=['title','episode','best_comment','comment','comment_num']) # 댓글
# 정보 모으고 평균 평점과 각 에피소드 평점 비교 후 semtiment 추가 작업하여 넣기

In [31]:
# 연재중 웹툰 클릭 후, 인기 순으로 정렬하기
def day_popular(day): # day는 2~8 / 2:월 3:화 4:수 5:목 6:금 7:토 8:일
    # 연재중 웹툰 클릭하기
    driver.find_element_by_xpath(f'//*[@id="wrap"]/header/div[3]/nav/ul/li[{day}]/a').click()
    time.sleep(1) # seconds 기다리기
    # 인기순으로 정렬
    driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/button[1]').click()
    time.sleep(1) # seconds 기다리기

In [32]:
# 무료 작품인지 아닌지 확인한 후 기다리면 무료작품일 경우는 pass 시키기
def free(num):
    di = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/a/div')
    find_i = di.find_elements_by_tag_name("i")
    return len(find_i)

In [33]:
# 페이지 끝까지 스크롤 다운하기
def scroll_down():
    last_height = driver.execute_script("return document.body.scrollHeight") # 스크롤 높이 가져오기

    # 끝까지 스크롤 다운 한번만 진행
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)

In [34]:
# 해당 작품으로 이동하기
def move_webtoon(num):
    driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/a').click()
    time.sleep(3)

In [35]:
# 웹툰의 글/그림/원작 정보 추출
def writer_info(num):
    # 글/그림/원작 정보의 xpath위치가 다른 경우 # //*[@id="content"]/div[1]/ul/li[{num}/div/div[1]
    try:
        txt = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/div/div[1]/a').text
        txt = txt.split(' / ')
    except:
        txt = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/div/a[2]').text
        txt = txt.split(' / ')
        
    if len(txt) == 1: # 글/그림이 같은 경우
        author = txt[0]
        artist = txt[0]
        origin = 'X'
    elif len(txt) == 2: # 글/그림이 다른 경우
        author = txt[0]
        artist = txt[1]
        origin = 'X'
    elif len(txt) == 3: # 원작까지 있는 경우 경우
        author = txt[0]
        artist = txt[1]
        origin = txt[2]
    return author, artist, origin

In [36]:
def change_day(day):
    if day == 2:
        return '월요일'
    elif day == 3:
        return '화요일'
    elif day == 4:
        return '수요일'
    elif day == 5:
        return '목요일'
    elif day == 6:
        return '금요일'
    elif day == 7:
        return '토요일'
    elif day == 8:
        return '일요일'

## 웹툰 information

'seq','day','title','descript','genre','author','artist','origin','total','star'
- 'seq': 임의의 번호
- 'day': 요일
- 'title': 웹툰 제목
- 'descript': 웹툰 소개 글
- 'genre': 태그된 장르
- 'author': 글
- 'artist': 그림
- 'origin': 원작작가 (없는 경우 X로 입력)
- 'total': 전체 웹툰 수 (예고편, 후기 등 모두 포함한 수)
- 'star': 웹툰의 전체 평균 평점

## 각 웹툰 에피소드 당 평점

'title','day','episode','score','date'
- 'title': 웹툰 제목
- 'day': 요일
- 'episode': 회차 이름
- 'score': 각 회차 당 score
- 'date': 회차 업로드 된 날짜

In [49]:
webtoon_info = pd.DataFrame(columns=['seq','day','title','descript','genre','author','artist','origin','total','star']) # 웹툰 정보
score_data = pd.DataFrame(columns=['title','day','episode','score','date']) # 에피소드 평점

In [50]:
# 각 에피소드 정보 모으기
def main_info_score(num,day): # num: Top 작품 수 지정, day:요일 지정
    global webtoon_info, score_data
    
    day_popular(day) # 완결 웹툰을 인기순으로 정렬하기
    c_day = change_day(day)
            
    free_y_n = free(num) # 무료작품인지 확인
    if free_y_n == 0: # 무료 작품인 경우
        title = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/div/a[1]').text
        # 글/그림/원작 & 전체 작품 수정보 추출
        author, artist, origin = writer_info(num)
        # 평균 평점
        star = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/ul/li[{num}]/div/div/span/span').text
        time.sleep(1)
        
        move_webtoon(num) # 해당 웹툰으로 이동하기
        
        # webtoon information
        # 작품 설명 # //*[@id="content"]/div[1]/div/div[2]/p
        descript = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/p').text
        # 장르 뽑기 # //*[@id="content"]/div[1]/div/div[2]/div/div
        genre = driver.find_element_by_xpath('//*[@id="content"]/div[1]/div/div[2]/div/div').text
        
        #전체 작품 수
        try:
            total = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[1]/div[1]').text.split(' ')[1][:-1]
        except:
            total = driver.find_element_by_xpath(f'//*[@id="content"]/div[4]/div[1]/div[1]').text.split(' ')[1][:-1]

        # 정보 저장하기
        webtoon_info.loc[len(webtoon_info)] = [num,c_day,title, descript, genre, author, artist, origin, total, star]
        time.sleep(1)
        ###########################################################################

        # 각 에피소드의 정보
        b_num = 3 # 중간에 광고가 있는 경우, 버튼의 위치가 다른 경우 때문에 추가
        try:
            driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/div[1]/div[2]/button[2]').click() # 1화부터
        except:
            b_num = 4
            driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/div[1]/div[2]/button[2]').click()
        
        scroll_down() # 스크롤 다운
        time.sleep(2)
        
        move = 1 # 정지하기 위한 조건
        while move==1:
            div = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/div[2]')
            button = div.find_elements_by_tag_name("button") # 다음 페이지로 이동할 수 있는 버튼 수 구하기

            if len(button)==12: # 이전편, 10개 목록, 다음편 으로 총 12개인경우
                move = 1 # 다음 목록으로도 이동할 수 있도록 계속 반복
                for n in range(2,12): # 2~11까지 10개 목록 반복
                    driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/div[2]/button[{n}]').click()
                    time.sleep(2)
                    # 각 정보 뽑기
                    ul = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul')
                    li = ul.find_elements_by_tag_name("li")
                    for num in range(1,len(li)+1):
                        episode = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul/li[{num}]/a/div[2]/p/span').text
                        score = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul/li[{num}]/a/div[2]/div/span[1]/span').text
                        date = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul/li[{num}]/a/div[2]/div/span[2]').text
                        score_data.loc[len(score_data)] = [title, c_day, episode, score, date]
                
                # 더이상 다음으로 넘어갈 것이 없는 경우 button이 비활성화 되어있음
                if driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/div[2]/button[{12}]').is_enabled() == True:
                    driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/div[2]/button[{12}]').click() # 다음목록으로 넘어가기
                    time.sleep(2)
                else: # 비활성화 된 경우
                    move = 0
            else:
                move = 0
                for n in range(2,len(button)):
                    driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/div[2]/button[{n}]').click()
                    time.sleep(2)

                    # 각 정보 뽑기
                    ul = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul')
                    li = ul.find_elements_by_tag_name("li")
                    for num in range(1,len(li)+1):
                        episode = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul/li[{num}]/a/div[2]/p/span').text
                        score = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul/li[{num}]/a/div[2]/div/span[1]/span').text
                        date = driver.find_element_by_xpath(f'//*[@id="content"]/div[{b_num}]/ul/li[{num}]/a/div[2]/div/span[2]').text
                        score_data.loc[len(score_data)] = [title, c_day, episode, score, date]
    # 기다리면 무료 작품인 경우
    else:
        pass

In [14]:
for day in range(2,9):
    for num in range(1,16):
        main_info_score(num,day)

In [15]:
webtoon_info

,seq,day,title,descript,genre,author,artist,origin,total,star
0,2,월요일,뷰티풀 군바리,'여자도 군대에 간다면?'본격 여자도 군대 가는 만화!,#드라마\n#직업드라마\n#밀리터리,설이,윤성원,X,388,9.81
1,3,월요일,신의 탑,자신의 모든 것이었던 소녀를 쫓아 탑에 들어온 소년\n그리고 그런 소년을 시험하는 탑,#판타지\n#세계관\n#이능력배틀물\n#소년왕도물\n#먼치킨,SIU,SIU,X,572,9.88
2,4,월요일,퀘스트지상주의,"[외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!\n공부, 싸...",#드라마\n#게임판타지\n#학원물,박태준 만화회사,박태준 만화회사,X,86,9.80
3,5,월요일,똑 닮은 딸,"'우리 엄마가 살인마인 것 같다.' \n성적 우수, 품행 단정, 모범적인 자식인 길...",#스릴러\n#2020 지상최대공모전\n#서스펜스,이담,이담,X,93,9.98
4,6,월요일,백수세끼,백수 시절 내 곁을 지켜줬던 그녀... 돌아와 주면 안 되겠니?\n음식 메뉴마다 담...,#드라마\n#음식&요리\n#감성드라마,치즈,치즈,X,159,9.84
...,...,...,...,...,...,...,...,...,...,...
89,11,일요일,나 혼자 특성빨로 무한 성장,평생 헌터가 되지 못할 줄 알았다.\n기회 같은 것은 찾아오지 않을 거라 여겼다.\...,#액션\n#레드아이스 스튜디오\n#헌터물\n#먼치킨\n#소설원작,샤이아탄,2사랑,선운(鮮雲),32,9.07
90,12,일요일,자매전쟁,"단 하나로 인정받고 싶었던 두 소녀, 예술고등학교 1등과 2등을 앞다투는 '원리온'...",#드라마\n#계략여주\n#학원로맨스\n#복수극,기맹기,기맹기,X,46,9.95
91,13,일요일,천화서고 대공자,넉넉한 풍채가 매력적인 무림맹주 후공은 \n어느 날 느닷없이 웬 비루먹을 청년의 몸...,#무협/사극\n#소설원작,128,MIDNIGHT STUDIO,김현영,25,9.91
92,14,일요일,소녀재판,학교에서 아무것도 아니었던 소녀의 예측불허 재판이 시작된다!\n평범한 박가을은 소위...,#드라마\n#2019 지상최대공모전\n#공감성수치,루즌아,보로콤,X,158,9.88


In [16]:
score_data

,title,day,episode,score,date
0,뷰티풀 군바리,월요일,예고편,9.54,15.02.15
1,뷰티풀 군바리,월요일,1화_신체 검사,9.54,15.02.22
2,뷰티풀 군바리,월요일,2화_입대,9.66,15.03.01
3,뷰티풀 군바리,월요일,3화_입대 첫 날,9.63,15.03.08
4,뷰티풀 군바리,월요일,4화_각개 전투,9.76,15.03.15
...,...,...,...,...,...
12568,판사 이한영,일요일,127화,9.88,4일 후 무료
12569,판사 이한영,일요일,128화,9.93,11일 후 무료
12570,판사 이한영,일요일,129화,9.83,18일 후 무료
12571,판사 이한영,일요일,130화,9.92,25일 후 무료


In [17]:
webtoon_info.to_csv('collected_data/continue_webtoon_info.csv', index=False)
score_data.to_csv('collected_data/continue_score_data.csv', index=False)

In [ ]:
# 추가 정보
# webtoon_info.to_csv('collected_data/continue_webtoon_info_plus.csv', index=False)
# score_data.to_csv('collected_data/continue_score_data_plus.csv', index=False)

## 웹툰 별, 각 에피소드 댓글

각 회차 당
- best 댓글 15개
- 최신 댓글 45개

In [ ]:
# comment_data

'title','day','episode','best_comment','comment'
- 'title': 웹툰 제목
- 'dat': 요일
- 'episode': 각 에피소드 명
- 'best_comment': best 댓글
- 'comment': 댓글
- 'comment_num': 총 댓글 수

In [12]:
comment_data = pd.DataFrame(columns=['title','episode','best_comment','comment','comment_num'])

베스트 댓글 수집 시, best댓글 클릭할 때 에러 발생  
.click()으로 하면 error발생 -> .send_keys(Keys.ENTER)로 대체

In [17]:
# 베스트 댓글 수집 (15개)
def best_comment():
    driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_sort_option_tab1"]').send_keys(Keys.ENTER) # best 댓글 클릭
    time.sleep(2)
    # 베스트 댓글이 15개가 아닐 수도 있으므로 수만큼 반복하도록 댓글 수 세기
    ul = driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul')
    li = ul.find_elements_by_tag_name("li")
    
    best_comment = ''
    for n in range(1,len(li)+1):
        txt = driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{n}]/div[1]/div/div[2]/span[2]').text+'\n'
        best_comment += txt
    return best_comment

In [18]:
# 전체 댓글 수집 (최신으로부터 300개 -> 더보기 20번 클릭)
def total_comment():
    driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_sort_option_tab2"]').click() # 전체 댓글 클릭
    time.sleep(2)
    # 전체 댓글 수가 적은 경우 미만으로만 수집하도록 하기
    repeat_num = 20 # 기본적으로 20번 모으도록 설정 20X15=300개의 댓글 수집
    comment_num = int(driver.find_element_by_xpath(f'//*[@id="cbox_module"]/div/div[1]/span').text.replace(',',''))
    if comment_num < repeat_num*15: # 모으려는 댓글 수보다 실제 댓글 수가 적을 경우
        repeat_num = comment_num//15 # 댓글 수 이내로 반복하여 수집할 수 있도록 하기
    if repeat_num == 0:
        repeat_num = 1
    
    n = 0
    total_comment = ''
    try:
        for repeat in range(repeat_num): # 15개 댓글 20번 모으기
            for fifteen in range(15): # 15번 반복
                n += 1 # 댓글 위치
                txt = driver.find_element_by_xpath(f'//*[@id="cbox_module_wai_u_cbox_content_wrap_tabpanel"]/ul/li[{n}]/div[1]/div/div[2]/span[1]').text+'\n'
                total_comment += txt
            driver.find_element_by_xpath(f'//*[@id="cbox_module"]/div/div[7]/a/span').click() # 더보기 클릭
            time.sleep(2)
            scroll_down() # 스크롤 다운
            time.sleep(2)
    except:
        # print('삭제된 댓글이 있습니다.')
        pass
    return total_comment, comment_num

In [19]:
# 각 에피소드 별 댓글 모으기
def main_comment(num,day):
    global comment_data
    
    day_popular(day) # 완결 웹툰을 인기순으로 정렬하기
    c_day = change_day(day)
            
    free_y_n = free(num) # 무료작품인지 확인
    if free_y_n == 0: # 무료 작품인 경우
        move_webtoon(num) # 해당 웹툰으로 이동하기
        
        # 웹툰 이름
        title = driver.find_element_by_xpath(f'//*[@id="content"]/div[1]/div/h2').text
        # 전체 회차 수 구하기
        try: # 중간 광고로 인한 위치가 다른 경우
            total_num = driver.find_element_by_xpath(f'//*[@id="content"]/div[3]/div[1]/div[1]').text.split(' ')[1][:-1]
        except:
            total_num = driver.find_element_by_xpath(f'//*[@id="content"]/div[4]/div[1]/div[1]').text.split(' ')[1][:-1]

        # 첫 에피소드로 이동
        driver.find_element_by_xpath(f'//*[@id="content"]/div[2]/div/a').click()
        time.sleep(2)
        scroll_down() # 제일 아래로 스크롤 다운
        
        try:
            for i in range(int(total_num)): # 에피소드 반복
                episode = driver.find_element_by_xpath(f'//*[@id="subTitle_toolbar"]').text
                #print(episode)
                # best 댓글 모으기
                best_txt = best_comment()
                # 전체 댓글 모으기
                total_txt, total_num = total_comment()

                comment_data.loc[len(comment_data)] = [title, episode, best_txt, total_txt, total_num]
                
                # 다음 에피소드로 이동
                # 음악이 삽입된 경우와 아닌 경우 나누기
                if driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').text == '다음화':
                    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').click()
                else:
                    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[5]/button').click()
            # 웹툰 목록으로 돌아가기
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)
                
        except:
            # 유료회차 창 닫기
            driver.find_element_by_xpath(f'/html/body/div[9]/div/div/div/button').send_keys(Keys.ENTER)
            # 웹툰 목록으로 돌아가기
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)

    # 기다리면 무료 작품인 경우
    else:
        pass

In [125]:
# 전체 요일 반복 & 요일 속 인기 웹툰 15개
for day in range(2,9):
    for num in range(1,16):
        main_comment(num,day)

예고편
1화_신체 검사
2화_입대
3화_입대 첫 날
4화_각개 전투


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[9]/div/div/div/button"}
  (Session info: chrome=112.0.5615.140)
Stacktrace:
Backtrace:
	(No symbol) [0x0068DCE3]
	(No symbol) [0x006239D1]
	(No symbol) [0x00534DA8]
	(No symbol) [0x0056019F]
	(No symbol) [0x005603AB]
	(No symbol) [0x0058EE62]
	(No symbol) [0x0057AF14]
	(No symbol) [0x0058D57C]
	(No symbol) [0x0057ACC6]
	(No symbol) [0x00556F68]
	(No symbol) [0x005580CD]
	GetHandleVerifier [0x00903832+2506274]
	GetHandleVerifier [0x00939794+2727300]
	GetHandleVerifier [0x0093E36C+2746716]
	GetHandleVerifier [0x00736690+617600]
	(No symbol) [0x0062C712]
	(No symbol) [0x00631FF8]
	(No symbol) [0x006320DB]
	(No symbol) [0x0063C63B]
	BaseThreadInitThunk [0x769F7D49+25]
	RtlInitializeExceptionChain [0x7736B74B+107]
	RtlClearBits [0x7736B6CF+191]


In [29]:
comment_data

,title,episode,best_comment,comment,comment_num
0,뷰티풀 군바리,예고편,진짜사나이하기전부터나온거임 오해하시는분없기를바람\n베도부터 본사람 좋아요\n길만들자...,클린봇이 부적절한 표현을 감지한 댓글입니다.\n으왕 정주행 시작!\n자대오고 격리당...,25949
1,뷰티풀 군바리,1화_신체 검사,진심 성폭행범 내탓아니다 니탓이다하는거 극혐\n성폭행 해놓고 다 여자가 짧게입은탓이...,지금 봤는데 나쁜놈 헤어질때부터 옆에 있었네\n한국에서 남자로 태어나는건 사실상 조...,29204
2,뷰티풀 군바리,2화_입대,내일 3월 2일자 해군 입대 입니다. 모두 축하해 주세요 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ...,지금 다시보니 후반부에 애진이 있네\n이작품이 9년연재할줄은 진짜 몰랏고 네이버 간...,19906
3,뷰티풀 군바리,3화_입대 첫 날,솔직히 남자로서 진짜 여자가 군대 가길 바라진 않는다. 하지만 적어도 나라 지키는 ...,주인공이 몸메가 좋고 가슴은c컵정도 되는것같아욘 주인공 같은시람이랑 하면 1시간 쌉...,16222
4,뷰티풀 군바리,4화_각개 전투,오빠가 진짜 쓰레기넼ㅋㅋㅋㅋ 숨 크게쉬랰ㅋㅋㅋ\n저도 군대에다녀왓습니다 참고로여자ㅎ...,저거 체질적으로 못느끼는 타입말고 다 콧물 범벅됨\n저거 마시면 진짜 뿅감 ㅋㅋㅋㅋ...,9362
...,...,...,...,...,...
100,뷰티풀 군바리,99화_11월,우지영은 왜 수아 질투하냐?ㅋㅋㅋ 폐급 쓰레기 이경들중에 가장 엘리트인 수아가 라시...,이제보니 오정화 옷 색깔 버그걸렸넹\n오정화 전역 후에도 정신과 상담 받게 한 장본...,3373
101,뷰티풀 군바리,100화_11월 2,수아가 한복입고 있는 장면에서 움찔한 사람 조용히 손들어라\n정수아 저러다가 군대안...,"솔직히 김가을은 육근옥이랑 맞먹음..,,\n주희린도나쓰나 허정인찡긋은 다시봐도 충격...",7166
102,뷰티풀 군바리,101화_11월 3,라시현ㅋㅋ 1소대에서 처리해야될걸 2소대 쳐맞고오라네 ㅋㅋㅋㅋ사실 라시현은 타종족이...,김가을 저걸 그냥 내보낸다고?\n소수라는 사람이 두달 된 이경한테 다른 소대 가서 ...,4783
103,뷰티풀 군바리,102화_11월 4,수아를 좋아한다고 고백한 송미남 상경님을 응원합니다\n라폭도들아.. 저게 라시현 본...,류다희 말이 맞음 뻔히 모두가 원하는 라시현중수행 이 있는데 지 혼자 감상에 젖어 ...,4573


In [38]:
# 내보내기
# comment_data.to_csv(f'collected_data/continue_comment_data_{day}_{num}.csv', index=False)

### 중간에 멈출경우

In [36]:
# 한 웹툰만 지정해서 실행
total_num = 388 # 전체 에피소드 화수 적기
title = '뷰티풀 군바리' # 웹툰 이름 적기

try:
    for i in range(109,int(total_num)): # 에피소드 반복 # 숫자는 현재까지 크롤링된 에피소드 이후 번호 적으면 됨
        episode = driver.find_element_by_xpath(f'//*[@id="subTitle_toolbar"]').text
        #print(episode)
        # best 댓글 모으기
        best_txt = best_comment()
        # 전체 댓글 모으기
        total_txt, total_num = total_comment()

        comment_data.loc[len(comment_data)] = [title, episode, best_txt, total_txt, total_num]

        # 다음 에피소드로 이동
        # 음악이 삽입된 경우와 아닌 경우 나누기
        if driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').text == '다음화':
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[4]/button').click()
        else:
            driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/div[2]/span[5]/button').click()
    # 웹툰 목록으로 돌아가기
    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)

except:
    # 유료회차 창 닫기
    driver.find_element_by_xpath(f'/html/body/div[9]/div/div/div/button').send_keys(Keys.ENTER)
    # 웹툰 목록으로 돌아가기
    driver.find_element_by_xpath(f'//*[@id="wrap"]/div[2]/div[1]/a').send_keys(Keys.ENTER)


In [35]:
comment_data

,title,episode,best_comment,comment,comment_num
0,뷰티풀 군바리,예고편,진짜사나이하기전부터나온거임 오해하시는분없기를바람\n베도부터 본사람 좋아요\n길만들자...,클린봇이 부적절한 표현을 감지한 댓글입니다.\n으왕 정주행 시작!\n자대오고 격리당...,25949
1,뷰티풀 군바리,1화_신체 검사,진심 성폭행범 내탓아니다 니탓이다하는거 극혐\n성폭행 해놓고 다 여자가 짧게입은탓이...,지금 봤는데 나쁜놈 헤어질때부터 옆에 있었네\n한국에서 남자로 태어나는건 사실상 조...,29204
2,뷰티풀 군바리,2화_입대,내일 3월 2일자 해군 입대 입니다. 모두 축하해 주세요 ㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠㅠ...,지금 다시보니 후반부에 애진이 있네\n이작품이 9년연재할줄은 진짜 몰랏고 네이버 간...,19906
3,뷰티풀 군바리,3화_입대 첫 날,솔직히 남자로서 진짜 여자가 군대 가길 바라진 않는다. 하지만 적어도 나라 지키는 ...,주인공이 몸메가 좋고 가슴은c컵정도 되는것같아욘 주인공 같은시람이랑 하면 1시간 쌉...,16222
4,뷰티풀 군바리,4화_각개 전투,오빠가 진짜 쓰레기넼ㅋㅋㅋㅋ 숨 크게쉬랰ㅋㅋㅋ\n저도 군대에다녀왓습니다 참고로여자ㅎ...,저거 체질적으로 못느끼는 타입말고 다 콧물 범벅됨\n저거 마시면 진짜 뿅감 ㅋㅋㅋㅋ...,9362
...,...,...,...,...,...
104,뷰티풀 군바리,103화_11월 5,정화 웃는 거 너무 예쁘다.. 이제 좀 행복했으면 ㅠㅠ\n정화 꽃길만 걷자ㅠㅠ 🌸 ...,그걸 못참고라니ㅋㅋㅋ 기수열외 못견디게 만든 게 누군데\n걍 289에서 수아랑 같이...,5234
105,뷰티풀 군바리,뷰티풀 군바리 인기투표 결과 발표,본격 주인공보다 류다희가 인기많은 웹툰\n이분들은 쉬는거 인정한다 쉬어도 된다\n류...,오로라 112는 진짜 뭐냐...?\n클린봇이 부적절한 표현을 감지한 댓글입니다.\n...,5071
106,뷰티풀 군바리,104화_11월 6,오오\n유예리 크게 한방 터뜨리지 않을까..? 쥐 죽은 듯이 조용히 넘어갈 타입은 ...,차라리 김소리가 엿들었어야지ㅋㅋㅋㅋ 진짜 이거만 생각하면 답답하네\n표지 행하나 ㅈ...,8725
107,뷰티풀 군바리,105화_11월 7,"제일나쁜사람은 중대장임 사실...\n김가을 : 너, 도대체 정체가 뭐야! / 유예리...",클린봇이 부적절한 표현을 감지한 댓글입니다.\n악마를 보았다....\n민지선 설유라...,6286


In [42]:
# 일부 요일, 한 웹툰만 지정해서 실행
comment_data = pd.DataFrame(columns=['title','episode','best_comment','comment','comment_num'])

day = 2
num = 3

main_comment(num,day)

# comment_data.to_csv(f'collected_data/continue_comment_data_{day}_{num}.csv', index=False)